In [1]:
"""
================================================================================
Nombre del Script: guardar_csv_bronze.py
Autor: Félix Cárdenas
Fecha de Creación: 2025-05-08
Última Modificación: 2025-05-08
Versión: 1.0.0

Descripción:
Este script forma parte de la capa BRONZE del proyecto BigData_Project.
Su propósito es guardar el dataset de pacientes en formato CSV en el bucket 
`dev-bronze` de MinIO, usando Pandas y Boto3.

Dependencias:
- Python >= 3.8
- Librerías: pandas, boto3, dotenv
"""

'\n================================================================================\nNombre del Script: guardar_csv_bronze.py\nAutor: Félix Cárdenas\nFecha de Creación: 2025-05-08\nÚltima Modificación: 2025-05-08\nVersión: 1.0.0\n\nDescripción:\nEste script forma parte de la capa BRONZE del proyecto BigData_Project.\nSu propósito es guardar el dataset de pacientes en formato CSV en el bucket \n`dev-bronze` de MinIO, usando Pandas y Boto3.\n\nDependencias:\n- Python >= 3.8\n- Librerías: pandas, boto3, dotenv\n'

In [2]:
# ================================================================================
# PASO 1: IMPORTACIÓN DE LIBRERÍAS
# ================================================================================
import os
import logging
from io import BytesIO
from datetime import datetime
from dotenv import load_dotenv
import boto3
import pandas as pd

In [3]:
# ================================================================================
# PASO 2: CARGA DE VARIABLES DE ENTORNO
# ================================================================================
load_dotenv("/home/jovyan/.env")

MINIO_ENDPOINT     = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS_KEY   = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET_KEY   = os.getenv("MINIO_ROOT_PASSWORD")
BUCKET_BRONZE      = os.getenv("MINIO_BUCKET_BRONZE")

ORIGEN             = "local"
DOMINIO            = "pacientes"
timestamp          = datetime.now().strftime("%Y%m%d%H%M")
archivo_csv        = f"origen_pacientes_{timestamp}.csv"
key_minio          = f"{ORIGEN}/{DOMINIO}/{archivo_csv}"
ruta_csv_local     = "/home/jovyan/datos/csv/pacientes_crudo.csv"

In [4]:
# ================================================================================
# PASO 3: LOGGING
# ================================================================================
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [5]:
# ================================================================================
# PASO 4: LECTURA DEL CSV LOCAL Y CONVERSIÓN
# ================================================================================
logger.info("Leyendo el archivo CSV local...")
df = pd.read_csv(ruta_csv_local)
logger.info(f"Archivo leído correctamente con {len(df)} registros.")

2025-05-09 17:12:40,554 - INFO - Leyendo el archivo CSV local...
2025-05-09 17:12:41,250 - INFO - Archivo leído correctamente con 1000000 registros.


In [6]:
# ================================================================================
# PASO 5: GUARDADO EN MinIO COMO CSV
# ================================================================================
logger.info(f"Conectando a MinIO y guardando en s3://{BUCKET_BRONZE}/{key_minio}...")
s3 = boto3.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

buffer = BytesIO()
df.to_csv(buffer, index=False)
buffer.seek(0)

s3.upload_fileobj(buffer, BUCKET_BRONZE, key_minio)
logger.info("✅ Archivo guardado exitosamente en MinIO.")

2025-05-09 17:12:41,260 - INFO - Conectando a MinIO y guardando en s3://dev-bronze/local/pacientes/origen_pacientes_202505091712.csv...
2025-05-09 17:12:42,885 - INFO - ✅ Archivo guardado exitosamente en MinIO.
